# Datatsets  

To be used for training, validation, and test data. They need to be the cleanest of the clean.  

I have a table of them that I will upload also, I just don't love the tables in markdown (and it's a fairly lopsided, ugly table anyway). To be uploaded when I find all the papers I need to reference.  


## Phyla  

#### Porifera

Sponges, as usual, are difficult to come by. Even more so since Compagen doesn't exist anymore.  

I looked for Sycon, because I know it's supposed to be one of the better ones, but since they took the Compagen site down, it is harder to find. I think I did at this link: https://datadryad.org/stash/dataset/doi:10.5061/dryad.tn0f3. But if I can't find the reads (and it doesn't look like they were ever posted anywhere) it won't be much good. So there's that.  

Right now I have these:  
1. Amphimedon queenslandica  
2. Ephydatia meulleri  


#### Ctenophora  

Ctenos are also pretty scarce. I contacted Joe to get some reads for Mnemiopsis, and he has some others as well, so I'll get what I can from there. 

Current ctenos:  
1. Mnemiopsis leidyi  
2. Pleurobrachia bachei
3. Beroe  

#### Cnidaria  

Definitely more cnidos to be had, so that's good. I discovered that the best way to search for datasets appears to be looking up genome assemblies on NCBI (if you go to the legacy genome assembly page, there is a place over on the right that says "FTP directory for GenBank assembly" (or it might say RefSeq sometimes) and if you click on that, it will take you to a page that looks like it was made in 1997 that is really just a directory full of resources. Find the one that ends in "genomic.fnz.gz" and right click to save the link. That's the only way I know to get the URL you need to curl.) and then looking up the raw read sets on ENA.  

I prioritized chromosome-level assemblies, but also took some scaffold ones. Also didn't pay too much attention to taxonomy, so closely related things were okay, and not having some groups represented was okay. I'm trying to get a good volume of reads, and I don't think it matters a whole lot that some of them come from the same genus.  

Current cnidos:  
1. Acropora hyacinthus  
2. Acropora millepora  
3. Cassiopea andromeda  
4. Aurelia aurita  
5. Diadumene lineata  

#### Mollusca  

For these guys, there seemed to be more species available, but fewer reads for each species. Hopefully it won't make a huge difference.  

Current mollusks:  
1. Achatina immaculata  
2. Archivesica marissinica  
3. Arion vulgaris  
4. Conus ventricosus  
5. Crassostrea gigas  
6. Dreissena polymorpha  
7. Gigantopelta aegis  
8. Lanistes nyassanus  
9. Margaritifera margaritifera  
10. Mercenaria mercenaria  
11. Pomacea canaliculata  
12. Ruditapes philippinarum  

## Processing  

These are the reads I'm going to start with for now. One of the best methods I know to get decontaminated reads is to map them onto a good assembly, so that's the goal for these sets. But before I do that, there are lots and lots of processing steps to take care of, so let's get to it.  

Most of this (minus naming basically) is happening from outside the directory that houses the actual data, here: `/scratch/genomics/spillanej/contamination`.  As I get more and more files, I will have to move things around so that I don't end up hitting my quota and cancelling all my jobs. Rebecca lined up some more space (5TB, unscrubbed) for me, here: `/store/public/spillanej`. I want to think about how I'll use it, but I'm okay for now.  

### Getting the reads and assemblies to a usable place  

### Genomes  

Genomes don't take up nearly the space that reads to, but I still want to be mindful of how many copies of them I have.  

Process:  
1. Download all genome files  
2. Rename to match Genus_species.fasta  
3. Format fastas to get rid of non-nucleotides  
4. Make the header names simple and informative(?)  
5. Index each genome for mapping  

#### Renaming

The assemblies all have names that are super similar. This is annoying. I have been renaming them as they download so that I don't get confused and end up with a bunch of incorrectly named genome assemblies. That would be a disaster. I have also been renaming the read files, either as soon as they download, or when I've downloaded all the ones I'm using from a particular platform and am catting them (or zcatting them) together.  

#### Formatting fastas  

Remember how the Actinia genome had some weird non-nucleotides in it? Well I do. And anvi'o had a script that solved that problem. Presumably other programs do too, but I already installed anvi'o and know how to use that script, so you can guess which program I decided to use for this. Am I sure that they all need this? No, definitely not. Some might be totally fine. But in the interest of treating them all the same way, I'm going to run it for all of them. It certainly won't hurt them if they are already good to go.  

`anvi-script-reformat-fasta datasets/Actinia_tenebrosa.fasta -o formatted_genomes/Actinia_tenebrosa.formatted.fasta --seq-type NT`  

This script also technically can change the headers so that they are simplified, but it doesn't seem to allow for much customization, so they don't end up very informative. As in, they would all be labeled the exact same way, so if I ever needed to identify what species we were talking about, it might be difficult. Would this kind of simplifying be fine for my purposes? Maybe, but since I'm not sure, I'm not going to risk it. See below for what I actually did.  

#### Simplifying the headers  

I don't know if I need to care about the read headers, but I'm hoping not. The assembly headers might not be super important after the mapping stage, but I still feel the need to fix them before I do that step, just in case. I guess working with Anvi'o has made me paranoid. So I used my tried-and-true but fairly janky way of making the headers in the genome assemblies simple and informative. From inside the directory that houses these (formatted_genomes), this is what this looks like:  

`awk '/^>/{print ">Genus_species_" ++i; next}{print}' Genus_species.formatted.fasta > ../simple_headers/Genus_species.fa`  

### Reads  

I want to treat all the read files the same too (according to platform) so even though they might not all need all the steps, I'm still going to do them for all files.  

1. Download read files  
2. Unzip read files  
3. Cat read files together according to platform  
4. Trim all reads (aggressive adapter trimming and nonexistent quality trimming)  
5. Map them to their respective genomes  
6. Save all those that map in new fastq files  

I'm going to have to see how large the mapped read files are, but my thought is that I'll be able to keep them in my /scratch/genomics space, and can transfer the trimmed read files (before mapping) and the bam files to my /store/public space. Zipped, of course. The original read files I'll delete, as I can just download them again if I ever need them again.  

#### Downloading   

I'm just using curl commands to get them all downloaded. Before, I was doing this on the login node, which is fine, but moving forward I'm going to pop these commands into scripts that I can submit as jobs.  

#### Samtools steps  

I was trying to figure out what samtools processing steps I need to go through, since these files take up a lot of space and I don't want to make ones that I don't really need. But I definitely need a bam file of all the reads that mapped to the genome, and I need the sorted version for blobtools to work.  

Rebecca mentioned that she has not had good luck with the filtering of these files. That sometimes the file sizes between the files at different stages of filtering are the same size. So I am doing a trial run on one of them (Acropora hyacinthus). I'm having it spit out another stage of sam file, one from after it should have filtered out all of the things that don't map. Hopefully they are different, and I can look at it enough to feel confident about the samtools filtering step.

### Snakemake  

I'm also working on learning Snakemake (here's some docs: https://snakemake.readthedocs.io/en/stable/index.html), so I'm hoping to do most of these things from in Snakemakes platform/format/style/syntax. I did one batch of genomes first, and got them as far as the genome indexing step, which was the first step I figured out how to do in Snakemake. I will definitely upload my Snakefile to github, but I'll also probably drop pieces of it in here as I make rules.  

### Space!  

I knew this was coming, but I have officially run out of space in my scratch/genomics directory! Good thing Rebecca made that other one for me. I'm going to transfer all of the raw reads there, at least until I get them trimmed. Then I will probably delete all the raw ones, and just keep the trimmed ones. I have all of the accessions recorded, so if I really need to get some or all of them back at some point, I can.  

I'm going to have to be a little strategic about this. The compute nodes of Hydra can't really interact with files in the /store/public/spillanej/ space, so I need to use it for storage and not as a place to keep things I'm interacting with more often. That, combined with the fact that these files take a while to move around means I'd rather keep them somewhere slightly more accessible, so I'm going to strategize what the best thing to do with my storage space is. I want it to be useful, and to keep things that I don't need right this minute, and that will free up some space to work in my scratch directories.  

ALSO, I found out that the way I catted (or rather, zcatted) the reads together is freaking out the process, so I'm going to have to redownload the ones I did that to. Everything in the first batch (Ephydatia muelleri, Pleurobrachia bachei, Nematostella vectensis, Cassiopea andromeda, Acropora hyacinthus, Acropora millepora, Actinia tenebrosa, Amphimedon queenslandica, Aurelia aurita) should be fine, since I unzipped those before catting them together, but all the ones I did second (Diadumene lineata (illumina reads only), Arion vulgaris, Crassostrea gigas, Dreissena polymorpha (pacbio reads only), Lanistes nyassanus, Margaritifera margaritifera, Mercenaria mercenaria, Pomacea canaliculata (illumina reads only), Ruditapes philippinarum) will need to be redownloaded. I actually don't have to do this for all of them, because if the read files were singular, I never catted them, and they are still fine (Diadumene lineata (pacbio reads only), Achatina immaculata, Archivesica marissinica, Conus ventricosus, Dreissena polymorpha (illumina reads only), Gigantopelta aegis, Pomacea canaliculata (pacbio reads only)). This is a bummer, because that took a while, but it will be an opportunity to automate more of the process, and it will help free up space in the short term. 

**Oof, this space thing is really coming for me**  
I keep having jobs that don't finish correctly because I'm hitting my quota. I didn't realize, but the /pool/genomics/spillanej/ space is different from my current space, so I'm going to start writing a ton of files there. And that's where I'll transfer all the sorted bam files and all of the reads that have been mapped to the genomes. This should give me much more breathing room than before.  